This is notebook with WandBi sweep that can be performed with this project FNN implementation. 

In [ ]:
#general libraries
import pandas as pd
import numpy as np
import copy
import wandb
#project code
from FNN import FNN
from OptimizersFunctions import *
from ActivFunctions import  *
from LossFunctions import *
from SuppFunctions import *
from TestingFunctions import *
from SweepFunctions import *

In [ ]:
#specifying WandBi data for sweep
api_key = ""#specify your api_key, but never share it
entity = "DL_project_Group_70"
project = "CIFAR"

In [ ]:
#loging to WandBi
wandb.login(api_key)

In [ ]:
#Specifying configuration for sweep
sweep_configuration = {
    "name": "sweepdemo",
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "dataset": {"values": ["CIFAR"]},#"MNIST","CIFAR"
        "epochs": {"values": [20,25,30]},
        "num_hidden_layers": {"values": [3, 4, 5]},
        "num_hidden_units": {"values": [ 64, 128, 256, 512, 1024]},
        "optimizer": {"values": ["sgd","sgd_momentum","rmsprop","nag","adam"]},
        "learning_rate": {"min": 0.0001, "max": 0.01},
        "grad_clip": {"min": 0.5, "max": 5.0},
        "momentum": {"min": 0.8, "max": 0.95},#sgd_momentum,nag
        "beta": {"min": 0.9, "max": 0.99},#rmsprop
        "beta1": {"min": 0.8, "max": 0.95},#adam
        "beta2": {"min": 0.96, "max": 0.99},#adam
        "batch_size": {"values": [ 64, 128, 256]},
        "l_coeff": {"min": 1e-5, "max": 1e-3},
        "l_method": {"values": ["none","l1", "l2"]},
        "weights_init": {"values": ["RandomUni", "RandomNor", "XavUni", "XavNor", "HeUni", "HeNor"]},
        "activation_hidden": {"values": ["identity", "sigmoid", "tanh", "relu", "leaky_relu"]},
        "activation_output": {"values": [ "softmax"]},
        "loss_function": {"values": [ "CrossEntropy", "SoftmaxCrossEntropy"]},
    },
}

In [ ]:
#setting sweep ID
sweep_id = wandb.sweep(sweep=sweep_configuration, project = project)

In [ ]:
#starting sweep
wandb.agent(sweep_id, function=main,count=2)